<a href="https://colab.research.google.com/github/kidrono/Kaggle-competitions/blob/main/kaggle_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle competition: Natural Language Processing with Disaster Tweets
The competition information could be found [here](https://www.kaggle.com/competitions/nlp-getting-started/overview)

In [8]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [10]:
! cp kaggle.json ~/.kaggle/

In [11]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle competitions download nlp-getting-started

  0% 0.00/593k [00:00<?, ?B/s]
100% 593k/593k [00:00<00:00, 145MB/s]


In [13]:
import zipfile
import os

if os.path.isfile("/data/train.csv"):
  print("allready exists")
else:
  zip_ref = zipfile.ZipFile("nlp-getting-started.zip", 'r')
  zip_ref.extractall("/data")
  zip_ref.close()

In [14]:
import pandas as pd
import numpy as np
import re
import torch

In [15]:
train_df = pd.read_csv("/data/train.csv")
train_labels = train_df['target']

In [16]:
one_hot_dict = dict()
i = 0
for text in train_df['text']:
  clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
  tweet = clean_tweet.split(' ')
  for word in tweet:
    word = word.lower()
    if word not in one_hot_dict.keys():
      one_hot_dict[word] = i
      i += 1
vec_dim = len(one_hot_dict) + 1
vec_dim

21675

In [17]:
def sentence_vector(text):
  vec = np.zeros(vec_dim)
  clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
  tweet = clean_tweet.split(' ')
  for word in tweet:
    word = word.lower()
    if word in one_hot_dict.keys():
      vec[one_hot_dict[word]] += 1
    else:
      vec[-1] += 1
  return vec

In [18]:
X = []
for text in train_df['text']:
  X.append(sentence_vector(text))
X = np.array(X)
X.shape

(7613, 21675)

In [19]:
class LogisticRegression(torch.nn.Module):

  def __init__(self, input_dim, classes) -> None:
    super().__init__()
    self.linear = torch.nn.Linear(input_dim, classes)
    self.sigmoid = torch.nn.Sigmoid()
  
  def forward(self, x):
    x = self.linear(x)
    x = self.sigmoid(x)
    return x

In [20]:
def train(x, y, model, loss, optimizer, ephocs):
  for epoch in epochs:
    y_pred = model(x)
    pred_loss = loss(y, y_pred)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    


In [22]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(penalty="l2").fit(X, train_labels)
y_prob= model.predict_proba(X)[:, 1]
y_prob

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([0.86471583, 0.9605333 , 0.89823862, ..., 0.90763466, 0.95452407,
       0.9931758 ])

In [23]:
from gensim.models import Word2Vec

In [24]:
tokens_list = []
i = 0
for text in train_df['text']:
  clean_tweet = re.sub(r'[^a-zA-Z ]+', '', text)
  tweet = clean_tweet.split(' ')
  tokens = []
  for word in tweet:
    word = word.lower()
    tokens.append(word)
  tokens_list.append(tokens)
train_df['tokens'] = tokens_list

train_df['tokens']

0       [our, deeds, are, the, reason, of, this, earth...
1           [forest, fire, near, la, ronge, sask, canada]
2       [all, residents, asked, to, shelter, in, place...
3       [, people, receive, wildfires, evacuation, ord...
4       [just, got, sent, this, photo, from, ruby, ala...
                              ...                        
7608    [two, giant, cranes, holding, a, bridge, colla...
7609    [ariaahrary, thetawniest, the, out, of, contro...
7610    [m, , utckm, s, of, volcano, hawaii, httptcozd...
7611    [police, investigating, after, an, ebike, coll...
7612    [the, latest, more, homes, razed, by, northern...
Name: tokens, Length: 7613, dtype: object

In [27]:
embeding = Word2Vec(train_df['text'][0].encode('utf-8').split(), min_count = 1,
                              vector_size = 300, window = 5)


In [28]:
train_df['text'][0].encode('utf-8').split()

[b'Our',
 b'Deeds',
 b'are',
 b'the',
 b'Reason',
 b'of',
 b'this',
 b'#earthquake',
 b'May',
 b'ALLAH',
 b'Forgive',
 b'us',
 b'all']